In [7]:
from torch import Tensor
import torch.nn.functional as f
import torch
import math
import torch.nn as nn
import math, copy, time

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def scaled_dot_product_attention(query, key, value, mask=None, dropout=None):
    scale = query.size(-1)
    temp = torch.matmul(query, key.transpose(-2, -1))/ math.sqrt(scale)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = f.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
#Should it be copies of the same layer?
class Attention(nn.Module):
    def __init__(self,d_model,h):
        super(Attention,self).__init__()
        self.d_k = d_model // h
        self.h = h
        self.linears = torch.clone(nn.Linear(d_model, d_model), 4)

In [ ]:
#Multi head attention u need to split query key and value into n vectors. Thse each are processed
#individually and are considered a head. Each produces an output that is concatenated to a single vector
#before going through the last linear layer. Theoretically, every head should learn something new

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: int, dim_in: int, dim_q: int, dim_k: int):
        super().__init__()
        self.heads = nn.ModuleList(
            [AttentionHead(dim_in, dim_q, dim_k) for _ in range(num_heads)]
        )
        self.linear = nn.Linear(num_heads * dim_k, dim_in)

    def forward(self, query: Tensor, key: Tensor, value: Tensor) -> Tensor:
        return self.linear(
            torch.cat([h(query, key, value) for h in self.heads], dim=-1)
        )

In [3]:
import torch

# Initialize the tensors with zeros
tensors = torch.rand(20)

# Reshape the tensors into a 4 by 5 array
tensors = tensors.reshape(4, 5)
tensors

tensor([[0.3399, 0.8518, 0.5645, 0.0181, 0.6281],
        [0.2697, 0.7004, 0.0772, 0.1434, 0.2014],
        [0.3485, 0.5635, 0.1693, 0.3000, 0.3270],
        [0.5904, 0.0124, 0.8236, 0.3627, 0.5030]])

In [ ]:
src = torch.rand(64, 32, 512)
tgt = torch.rand(64, 16, 512)
out = Transformer()(src, tgt)
print(out.shape)

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
        
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

def inference_test():
    test_model = make_model(11, 11, 2)
    test_model.eval()
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    memory = test_model.encode(src, src_mask)
    ys = torch.zeros(1, 1).type_as(src)

    for i in range(9):
        out = test_model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    print("Example Untrained Model Prediction:", ys)


def run_tests():
    for _ in range(10):
        inference_test()


show_example(run_tests)